# Import Bibliotek

In [9]:
import pandas as pd
import numpy as np
import random
import math
import copy

# Wczytanie Danych

In [10]:
df = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/WorkpieceFeatures-Python.xlsx")
narzedzie = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/dobor-narzedzia.xlsx")
obrabiarki = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/Obrabiarki.xlsx")
parametr_rt = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/Ra-Rt.xlsx")
parametr_cmc = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/CMC-kc1.xlsx")
obliczanie_vc = pd.read_excel("C:/Users/marci/anaconda3/Scripts/Notebook/Praca Inzynierska/Input/Par_vc_Val.xlsx")

In [12]:
df


,IDFtr,IDPart,FtrNo,FtrType,D,L,Ra,IT,A,V,...,QRG,QMT,QFN,QC,Dobl,Mat_PN,Mat_EN,Rm,HB,Rodzaj Obrobki
0,P0001F003,Part0001,#3,RFPOCL,38.0,80.0,0.63,6,3040.0,0.090729,...,3.1,0.55,0.22,3.87,48.87,St4S,S275JR,420,140,Obrobka Zgrubna
1,P0001F006,Part0001,#6,RFPOCL,40.0,33.0,1.25,8,1320.0,0.041469,...,3.1,0.55,0.22,3.87,48.87,St4S,S275JR,420,140,Obrobka Zgrubna
2,P0001F009,Part0001,#9,RFPOCL,45.0,72.0,0.63,6,3240.0,0.114511,...,3.1,0.55,0.22,3.87,48.87,St4S,S275JR,420,140,Obrobka Zgrubna
3,P0001F012,Part0001,#12,RFPOCL,40.0,48.0,1.25,8,1920.0,0.060319,...,3.1,0.55,0.22,3.87,48.87,St4S,S275JR,420,140,Obrobka Zgrubna
4,P0002F011,Part0002,#11,RFPOCL,86.0,95.0,0.63,6,8170.0,0.551836,...,5.2,0.90,0.23,6.33,101.33,40H,41Cr4,1000,217,Obrobka Zgrubna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,P0034F005,Part0034,#5,RFPOCL,72.0,15.0,5.00,12,1080.0,0.061073,...,3.6,0.75,0.24,4.59,111.59,15HGN,18NiCr5-4,830,217,Obrobka Ksztaltujaca
250,P0034F007,Part0034,#7,RFPOCL,60.0,47.0,0.80,6,2820.0,0.132889,...,3.6,0.75,0.24,4.59,111.59,15HGN,18NiCr5-4,830,217,Obrobka Ksztaltujaca
251,P0034F009,Part0034,#9,RFPOCL,50.0,61.0,0.80,6,3050.0,0.119773,...,3.6,0.75,0.24,4.59,111.59,15HGN,18NiCr5-4,830,217,Obrobka Ksztaltujaca
252,P0034F013,Part0034,#13,RFPOCL,48.0,30.0,5.00,12,1440.0,0.054287,...,3.6,0.75,0.24,4.59,111.59,15HGN,18NiCr5-4,830,217,Obrobka Ksztaltujaca


# Obliczenie mocy rzeczywistej obrabiarek + sortowanie

In [13]:
obrabiarki['Moc*sprawnosc'] = obrabiarki['Moc [kW]'] * obrabiarki['Sprawność']

In [14]:
obrabiarki = obrabiarki.sort_values(['Moc*sprawnosc'], ignore_index = True)

df = df.sort_values(by = ['IDPart', 'D', 'Rodzaj Obrobki'], ascending =[True, True, False], ignore_index = True)

# Dodanie kolumn potrzebnych w późniejszym etapie do df 

In [15]:
liczba_czesci = df['IDPart'].nunique()
lista_material = [4205, 4215, 4225, 4235, 1525, 2015, 1515, 2025, 12345]

df['Op_A'] = 'LT'
df['HT'] = 'R'
df['y0'] = -6
df['Kr'] = np.nan
df['MocxSprawnosc'] = np.nan
df['Prędkość n Max'] = np.nan
df['Gniazdo b'] = np.nan
df['Gniazdo h'] = np.nan
df['ap obrobki'] = np.nan
df['ap obliczone'] = np.nan
df['re'] = np.nan
df['Geometry_short'] = np.nan
df['CMC'] = np.nan
df['kc1'] = np.nan
df['mc'] = np.nan
df['Rt'] = np.nan
df['material'] = np.nan
df['Holder'] = np.nan
df['Geometry'] = np.nan
df['fn_z'] = np.nan
df['f1'] = np.nan
df['f2'] = np.nan
df['vc1'] = np.nan
df['vc2'] = np.nan
df['vc'] = np.nan
df['n_w'] = np.nan
df['Q'] = np.nan
df['hm'] = np.nan
df['kc'] = np.nan
df['Fc'] = np.nan
df['Pc'] = np.nan
df['tg'] = np.nan
df['indeks_w'] = np.nan
df['check_Pc'] = np.nan


In [16]:
max_index_obrabiarki = obrabiarki.shape[0]        # Odczytanie maksymalnego indeksu dla obrabiarek
max_index_df = df.shape[0]-1                        # Odczytanie maksymalnego indeksu dla df
max_indeks_cmc = parametr_cmc.shape[0]-1            # Odczytanie maksymalnego indeksu dla parametr_cmc
max_indeks_ra_rt = parametr_rt.shape[0]-1           # Odczytanie maksymalnego indeksu dla parametr_rt

# Przypisanie kątów, ap oraz re dla danej obróbki


In [17]:
mozliwe_katy_obrobka_zgrubna = [45, 60, 75]
mozliwe_katy_obrobka_ksztaltujaca = [91, 93, 95]

for indeks_df in range(0, max_index_df+1):
    if 'Obrobka Zgrubna' in df['Rodzaj Obrobki'][indeks_df]:
        df['Kr'][indeks_df] = random.choice(mozliwe_katy_obrobka_zgrubna)
        df['ap obrobki'][indeks_df] = df['QRRG'][indeks_df]/2  
        df['re'][indeks_df] = 1.2
        df['Geometry_short'][indeks_df] = '-PR'
    else: 
        df['Kr'][indeks_df] = random.choice(mozliwe_katy_obrobka_ksztaltujaca)
        df['ap obrobki'][indeks_df] = (df['DBMT'][indeks_df] - df['DBFN'][indeks_df])/2
        df['re'][indeks_df] = 0.8
        df['Geometry_short'][indeks_df] = '-PM'
        
df.sample(4)

<ipython-input-17-49d3de03a205>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kr'][indeks_df] = random.choice(mozliwe_katy_obrobka_zgrubna)
<ipython-input-17-49d3de03a205>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ap obrobki'][indeks_df] = df['QRRG'][indeks_df]/2
<ipython-input-17-49d3de03a205>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['re'][indeks_df] = 1.2
<ipython-input-17-49d3de03a205>:9: SettingWithCopy

,IDFtr,IDPart,FtrNo,FtrType,D,L,Ra,IT,A,V,...,vc,n_w,Q,hm,kc,Fc,Pc,tg,indeks_w,check_Pc
172,P0026F013,Part0026,#13,RFPOCL,50.0,38.0,0.63,6,1900.0,0.074613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,P0031F005,Part0031,#5,RFPOCL,92.0,300.0,0.63,6,27600.0,1.994283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,P0024F003,Part0024,#3,RFPOCL,20.0,20.0,0.63,6,400.0,0.006283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,P0018F012,Part0018,#12,RFPOCL,30.0,30.0,0.63,6,900.0,0.021206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Przypisanie współczynników CMC Kc1 mc

In [18]:
for indeks2_df in range(0, max_index_df+1):
    for indeks_cmc_table in range(0, max_indeks_cmc+1):
         if df['Mat_EN'][indeks2_df] in parametr_cmc['Mat_EN'].unique()[indeks_cmc_table]:
                df['CMC'][indeks2_df] = parametr_cmc['CMC'][indeks_cmc_table]
                df['kc1'][indeks2_df] = parametr_cmc['kc1'][indeks_cmc_table]               
                df['mc'][indeks2_df] = parametr_cmc['mc'][indeks_cmc_table]          

    for indeks_ra_rt_table in range(0, max_indeks_ra_rt+1):
         if df['Ra'][indeks2_df] == parametr_rt['Ra'][indeks_ra_rt_table]:
                df['Rt'][indeks2_df] = parametr_rt['Rt'][indeks_ra_rt_table]
                
df.sample(2)
        

<ipython-input-18-413212202b23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CMC'][indeks2_df] = parametr_cmc['CMC'][indeks_cmc_table]
<ipython-input-18-413212202b23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kc1'][indeks2_df] = parametr_cmc['kc1'][indeks_cmc_table]
<ipython-input-18-413212202b23>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mc'][indeks2_df] = parametr_cmc['mc'][indeks_cmc_table]
<ipython-inp

,IDFtr,IDPart,FtrNo,FtrType,D,L,Ra,IT,A,V,...,vc,n_w,Q,hm,kc,Fc,Pc,tg,indeks_w,check_Pc
93,P0015F013,Part0015,#13,RFPOCL,40.0,32.0,0.8,6,1280.0,0.040212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,P0029F007,Part0029,#7,RFPOCL,58.0,12.0,1.6,6,696.0,0.031705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dobór narzędzia oraz parametrów

In [19]:
IDPart = 0
for indeks3_df in range(0, max_index_df+1):
    if df['IDPart'][indeks3_df] == IDPart:
        df['MocxSprawnosc'][indeks3_df] = MocSprawnosc_Ost                                                     
        df['Prędkość n Max'][indeks3_df] = Prędkość_n_Max_obrabiarki_Ost
        df['Gniazdo b'][indeks3_df] = Gniazdo_b_Ost
        df['Gniazdo h'][indeks3_df] = Gniazdo_h_Ost
        df['material'][indeks3_df] = Material_Ost
        df['Holder'][indeks3_df] = Holder_Ost
        df['Geometry'][indeks3_df] = Geometry_Ost
        ap_obliczone_Ost = df['ap obrobki'][indeks3_df] / np.ceil(df['ap obrobki'][indeks3_df] / rslt_df1['ap_z'][random_index_rslt_df1])
        df['ap obliczone'][indeks3_df] = ap_obliczone_Ost
        if 'Obrobka Zgrubna' in df['Rodzaj Obrobki'][indeks3_df]:
            fn_z_Ost = rslt_df1['fn_z'][random_index_rslt_df1]                        
        else:
            fn_z_Ost = math.sqrt( (8 * df['re'][indeks3_df] * df['Rt'][indeks3_df])/ 1000)        
        df['fn_z'][indeks3_df] = fn_z_Ost
                  
    else:
        Pc_Check_Ost = 0       
        MocSprawnosc_Ost = 0                                                     
        Prędkość_n_Max_obrabiarki_Ost = 0
        Gniazdo_b_Ost = 0
        Gniazdo_h_Ost = 0        
        IDPart = df['IDPart'][indeks3_df]
        df_part = df.loc[
                        (df['IDPart'] == IDPart)
                        ]


        df_part = df_part.sort_values(['D'], ignore_index = True)

        for obrabiarka in range(0, max_index_obrabiarki+1):
            if obrabiarka == 13:
                df['MocxSprawnosc'][indeks3_df] = MocSprawnosc_Ost                                                     
                df['Prędkość n Max'][indeks3_df] = Prędkość_n_Max_obrabiarki_Ost
                df['Gniazdo b'][indeks3_df] = Gniazdo_b_Ost
                df['Gniazdo h'][indeks3_df] = Gniazdo_h_Ost
                df['material'][indeks3_df] = Material_Ost
                df['Holder'][indeks3_df] = Holder_Ost
                df['Geometry'][indeks3_df] = Geometry_Ost
                df['ap obliczone'][indeks3_df] = ap_obliczone_Ost
                df['fn_z'][indeks3_df] = fn_z_Ost
                
            else:
                    
                print("obrabiarka: ", obrabiarka)
                MocSprawnosc = obrabiarki['Moc*sprawnosc'][obrabiarka]
                Prędkość_n_Max_obrabiarki = obrabiarki['Prędkość n Max'][obrabiarka]
                Gniazdo_b = obrabiarki['Gniazdo b'][obrabiarka]
                Gniazdo_h = obrabiarki['Gniazdo h'][obrabiarka]

                for dobrany_material in range(0, len(lista_material)):
                    try:


                        rslt_df = narzedzie.loc[
                                        (narzedzie['Geometry_short'] == df_part['Geometry_short'][0]) &
                                        (narzedzie['kr'] == df_part['Kr'][0]) &
                                        (narzedzie['b'] == Gniazdo_b) &
                                        (narzedzie['h'] == Gniazdo_h) &            
                                        (narzedzie['HT'] == df_part['HT'][0]) &
                                        (narzedzie['Re'] == df_part['re'][0]) &
                                        (narzedzie['Material'] == lista_material[dobrany_material]) &
                                        (narzedzie['OP_A'] == df_part['Op_A'][0])
                                        ]

                        rslt_df = rslt_df.sort_values(['ap_z'], ignore_index = True, ascending=False)
                        max_indeks_rslt_df = rslt_df.shape[0]-1
                        rslt_df['Ilosc Przejsc'] = np.nan
                        for indeks1_rslt_df in range(0, max_indeks_rslt_df+1):
                            rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])

                        rslt_df1 = rslt_df.loc[(rslt_df['Ilosc Przejsc'] == rslt_df['Ilosc Przejsc'].unique().min())]
                        random_index_rslt_df1 = rslt_df1.sample(1).index[0] #wybor narzedzia

                        print(dobrany_material)

                        
                        ap_obliczone = df_part['ap obrobki'][0] / np.ceil(df_part['ap obrobki'][0] / rslt_df1['ap_max'][random_index_rslt_df1])
                        
                        Holder = rslt_df['Holder'][random_index_rslt_df1]
                        Geometry = rslt_df['Geometry'][random_index_rslt_df1]
                        Material = lista_material[dobrany_material]

                        if 'Obrobka Zgrubna' in df_part['Rodzaj Obrobki'][0]:
                            fn_z = rslt_df1['fn_z'][random_index_rslt_df1]                        
                        else:
                            fn_z = math.sqrt( (8 * df_part['re'][0] * df_part['Rt'][0])/ 1000)                    


                        rslt_df_vc = obliczanie_vc.loc[                            
                                (obliczanie_vc['CMC'] == df_part['CMC'][0]) &
                                (obliczanie_vc['Grade'] == Material) &
                                (obliczanie_vc['f1'] <= fn_z) &           
                                (obliczanie_vc['f2'] >= fn_z)
                                ]

                        rslt_df_vc = rslt_df_vc.sort_values(['CMC'], ignore_index = True, ascending=False)                       
                        vc1 = rslt_df_vc['Vc1'][0]
                        vc2 = rslt_df_vc['Vc2'][0]                                        
                        f1 = rslt_df_vc['f1'][0]
                        f2 = rslt_df_vc['f2'][0]                    

                        vc = ((vc2 - vc1) / (f2 - f1) * fn_z) + vc1 - ((vc2 - vc1) * f1) / (f2 - f1)     
                        n_w = (1000 * vc ) / (math.pi * df_part['DPF'][0])
                        if n_w < Prędkość_n_Max_obrabiarki:

 
                            hm = fn_z * math.sin(df_part['Kr'][0] * math.pi / 180)
                            kc = df_part['kc1'][0] * math.pow(hm, df_part['mc'][0]) * (1 - (df_part['y0'][0]/ 100 ))
                            Fc = kc * fn_z * ap_obliczone
                            Pc = Fc * vc / 60000      
                            Pc_Check = Pc / MocSprawnosc
                            print(Pc_Check)
                            if Pc > MocSprawnosc:
                                n_w = -1
                                f1 = -1
                                f2 = -1
                                vc1 = -1
                                vc2 = -1

                            elif Pc_Check > Pc_Check_Ost:
                                Pc_Check_Ost = Pc_Check
                                Pc_Ost = Pc
                                MocSprawnosc_Ost = MocSprawnosc                                                      
                                Prędkość_n_Max_obrabiarki_Ost = Prędkość_n_Max_obrabiarki
                                Gniazdo_b_Ost = Gniazdo_b
                                Gniazdo_h_Ost = Gniazdo_h
                                ap_obliczone_Ost = ap_obliczone
                                Holder_Ost = Holder
                                Geometry_Ost = Geometry
                                Material_Ost = Material
                                fn_z_Ost = fn_z
                                vc_Ost = vc   
                                n_w_Ost = n_w  
                                Q_Ost = Q 
                                hm_Ost = hm
                                kc_Ost = kc
                                Fc_Ost = Fc
                                Pc_Ost = Pc
                                #tg_Ost = tg


                            else:
                                continue     

                        else:
                            #predkosc wrzeciona jest za duza
                            continue










                    except:
                        # Brak narzedzia
                        continue


obrabiarka:  0
0
1
2
3
7
2.2751635172452165
obrabiarka:  1


<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

obrabiarka:  2
obrabiarka:  3
0
1
2
3
7
1.1375817586226082
obrabiarka:  4
0
1
2
3


<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

1.6997942274845954
7
1.0037486105493603
obrabiarka:  5
0
2.5538371804746327
1
1.8632083558300656
2
1.9457807099038478
3
1.2651414796508753
7
0.9201028930035803
obrabiarka:  6
0


...

<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

2
1.399649323618042
3
0.8546531263536755
7
0.5435397049784595
obrabiarka:  11
0
2.3456512452804534
1
1.696919976503542
2
1.7495616545225523
3
1.0683164079420941
7
0.5435397049784595
obrabiarka:  12


<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

0
1.3761153972311997
1
1.2444079827692642
2
1.283011879983205
3
0.6267456259926953
7
0.3985957836508703


<ipython-input-19-b1e2062b2b46>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Ilosc Przejsc'][indeks1_rslt_df] = np.ceil(df_part['ap obrobki'][0] / rslt_df['ap_z'][indeks1_rslt_df])
<ipython-input-19-b1e2062b2b46>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MocxSprawnosc'][indeks3_df] = MocSprawnosc_Ost
<ipython-input-19-b1e2062b2b46>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prędkość n Max'][indeks3_

In [20]:
for indeks3_df in range(0, max_index_df+1):

    rslt_df_vc = obliczanie_vc.loc[                            
            (obliczanie_vc['CMC'] == df['CMC'][indeks3_df]) &
            (obliczanie_vc['Grade'] == df['material'][indeks3_df]) &
            (obliczanie_vc['f1'] <= df['fn_z'][indeks3_df]) &           
            (obliczanie_vc['f2'] >= df['fn_z'][indeks3_df])
            ]

    rslt_df_vc = rslt_df_vc.sort_values(['CMC'], ignore_index = True, ascending=False)                       
    df['vc1'][indeks3_df] = rslt_df_vc['Vc1'][0]
    df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]                                        
    df['f1'][indeks3_df] = rslt_df_vc['f1'][0]
    df['f2'][indeks3_df] = rslt_df_vc['f2'][0] 


    df['vc'][indeks3_df] = ((df['vc2'][indeks3_df] - df['vc1'][indeks3_df]) / (df['f2'][indeks3_df] - df['f1'][indeks3_df]) * df['fn_z'][indeks3_df]) + df['vc1'][indeks3_df] - ((df['vc2'][indeks3_df] - df['vc1'][indeks3_df]) * df['f1'][indeks3_df]) / (df['f2'][indeks3_df] - df['f1'][indeks3_df])     
    df['n_w'][indeks3_df] = (1000 * df['vc'][indeks3_df] ) / (math.pi * df['DPF'][indeks3_df])
    if df['n_w'][indeks3_df] > df['Prędkość n Max'][indeks3_df]:
        df['indeks_w'][indeks3_df] = 1
    else:                       
        df['indeks_w'][indeks3_df] = 0

    df['Q'][indeks3_df] = df['vc'][indeks3_df] * df['fn_z'][indeks3_df] * df['ap obliczone'][indeks3_df]   
    df['hm'][indeks3_df] = df['fn_z'][indeks3_df] * math.sin(df['Kr'][indeks3_df] * math.pi / 180)
    df['kc'][indeks3_df] = df['kc1'][indeks3_df] * math.pow(df['hm'][indeks3_df], df['mc'][indeks3_df]) * (1 - (df['y0'][indeks3_df]/ 100 ))
    df['Fc'][indeks3_df] = df['kc'][indeks3_df] * df['fn_z'][indeks3_df] * df['ap obliczone'][indeks3_df]
    df['Pc'][indeks3_df] = df['Fc'][indeks3_df] * df['vc'][indeks3_df] / 60000      
    
    
    if df['Rodzaj Obrobki'][indeks3_df] == 'Obrobka Ksztaltujaca':
        df['tg'][indeks3_df] = round((df['L'][indeks3_df] * df['DBFN'][indeks3_df] * 60) / (318 * df['vc'][indeks3_df] * df['fn_z'][indeks3_df]),3)       
    else:
        df['tg'][indeks3_df] = round((df['L'][indeks3_df] * df['DBMT'][indeks3_df] * 60) / (318 * df['vc'][indeks3_df] * df['fn_z'][indeks3_df]),3)
       
    
    if df['Pc'][indeks3_df] > df['MocxSprawnosc'][indeks3_df]:
        df['check_Pc'][indeks3_df] = 1
    else:
        df['check_Pc'][indeks3_df] = 0


<ipython-input-20-a9e05d467139>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc1'][indeks3_df] = rslt_df_vc['Vc1'][0]
<ipython-input-20-a9e05d467139>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]
<ipython-input-20-a9e05d467139>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f1'][indeks3_df] = rslt_df_vc['f1'][0]
<ipython-input-20-a9e05d467139>:14: SettingWithCopyWarning: 

<ipython-input-20-a9e05d467139>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hm'][indeks3_df] = df['fn_z'][indeks3_df] * math.sin(df['Kr'][indeks3_df] * math.pi / 180)
<ipython-input-20-a9e05d467139>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kc'][indeks3_df] = df['kc1'][indeks3_df] * math.pow(df['hm'][indeks3_df], df['mc'][indeks3_df]) * (1 - (df['y0'][indeks3_df]/ 100 ))
<ipython-input-20-a9e05d467139>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

<ipython-input-20-a9e05d467139>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc1'][indeks3_df] = rslt_df_vc['Vc1'][0]
<ipython-input-20-a9e05d467139>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]
<ipython-input-20-a9e05d467139>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f1'][indeks3_df] = rslt_df_vc['f1'][0]
<ipython-input-20-a9e05d467139>:14: SettingWithCopyWarning: 

<ipython-input-20-a9e05d467139>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['check_Pc'][indeks3_df] = 0
<ipython-input-20-a9e05d467139>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc1'][indeks3_df] = rslt_df_vc['Vc1'][0]
<ipython-input-20-a9e05d467139>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]
<ipython-input-20-a9e05d467139>:13: SettingWithCopyWarning: 
A value is 

<ipython-input-20-a9e05d467139>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fc'][indeks3_df] = df['kc'][indeks3_df] * df['fn_z'][indeks3_df] * df['ap obliczone'][indeks3_df]
<ipython-input-20-a9e05d467139>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pc'][indeks3_df] = df['Fc'][indeks3_df] * df['vc'][indeks3_df] / 60000
<ipython-input-20-a9e05d467139>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tg'][indeks3

...

<ipython-input-20-a9e05d467139>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kc'][indeks3_df] = df['kc1'][indeks3_df] * math.pow(df['hm'][indeks3_df], df['mc'][indeks3_df]) * (1 - (df['y0'][indeks3_df]/ 100 ))
<ipython-input-20-a9e05d467139>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fc'][indeks3_df] = df['kc'][indeks3_df] * df['fn_z'][indeks3_df] * df['ap obliczone'][indeks3_df]
<ipython-input-20-a9e05d467139>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

<ipython-input-20-a9e05d467139>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hm'][indeks3_df] = df['fn_z'][indeks3_df] * math.sin(df['Kr'][indeks3_df] * math.pi / 180)
<ipython-input-20-a9e05d467139>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kc'][indeks3_df] = df['kc1'][indeks3_df] * math.pow(df['hm'][indeks3_df], df['mc'][indeks3_df]) * (1 - (df['y0'][indeks3_df]/ 100 ))
<ipython-input-20-a9e05d467139>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

<ipython-input-20-a9e05d467139>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]
<ipython-input-20-a9e05d467139>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f1'][indeks3_df] = rslt_df_vc['f1'][0]
<ipython-input-20-a9e05d467139>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f2'][indeks3_df] = rslt_df_vc['f2'][0]
<ipython-input-20-a9e05d467139>:17: SettingWithCopyWarning: 
A

<ipython-input-20-a9e05d467139>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc1'][indeks3_df] = rslt_df_vc['Vc1'][0]
<ipython-input-20-a9e05d467139>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc2'][indeks3_df] = rslt_df_vc['Vc2'][0]
<ipython-input-20-a9e05d467139>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f1'][indeks3_df] = rslt_df_vc['f1'][0]
<ipython-input-20-a9e05d467139>:14: SettingWithCopyWarning: 

<ipython-input-20-a9e05d467139>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f2'][indeks3_df] = rslt_df_vc['f2'][0]
<ipython-input-20-a9e05d467139>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vc'][indeks3_df] = ((df['vc2'][indeks3_df] - df['vc1'][indeks3_df]) / (df['f2'][indeks3_df] - df['f1'][indeks3_df]) * df['fn_z'][indeks3_df]) + df['vc1'][indeks3_df] - ((df['vc2'][indeks3_df] - df['vc1'][indeks3_df]) * df['f1'][indeks3_df]) / (df['f2'][indeks3_df] - df['f1'][indeks3_df])
<ipython-input-20-a9e05d467139>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [21]:
df
    

,IDFtr,IDPart,FtrNo,FtrType,D,L,Ra,IT,A,V,...,vc,n_w,Q,hm,kc,Fc,Pc,tg,indeks_w,check_Pc
0,P0001F003,Part0001,#3,RFPOCL,38.0,80.0,0.63,6,3040.0,0.090729,...,345.000003,2196.338236,774.870007,0.386370,1253.568489,2815.514826,16.189210,4.241,0.0,0.0
1,P0001F003,Part0001,#3,RFPOCL,38.0,80.0,0.63,6,3040.0,0.090729,...,488.789766,3111.732293,18.625392,0.138543,970.048035,36.963795,0.301125,8.518,0.0,0.0
2,P0001F006,Part0001,#6,RFPOCL,40.0,33.0,1.25,8,1320.0,0.041469,...,345.000003,2196.338236,326.025003,0.346410,1219.817277,1152.727326,6.628182,1.830,0.0,0.0
3,P0001F012,Part0001,#12,RFPOCL,40.0,48.0,1.25,8,1920.0,0.060319,...,345.000003,2196.338236,326.025003,0.282843,1159.533693,1095.759340,6.300616,2.661,0.0,0.0
4,P0001F006,Part0001,#6,RFPOCL,40.0,33.0,1.25,8,1320.0,0.041469,...,455.000002,2896.619975,25.025000,0.199726,1062.932595,58.461293,0.443331,2.737,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,P0034F009,Part0034,#9,RFPOCL,50.0,61.0,0.80,6,3050.0,0.119773,...,205.000001,567.421973,12.300000,0.159781,1139.294193,68.357652,0.233555,17.629,0.0,0.0
250,P0034F007,Part0034,#7,RFPOCL,60.0,47.0,0.80,6,2820.0,0.132889,...,145.000001,401.347252,261.048336,0.282843,1314.138186,2365.886780,5.717560,9.325,0.0,0.0
251,P0034F007,Part0034,#7,RFPOCL,60.0,47.0,0.80,6,2820.0,0.132889,...,205.000001,567.421973,12.300000,0.159976,1139.641471,68.378488,0.233627,16.287,0.0,0.0
252,P0034F005,Part0034,#5,RFPOCL,72.0,15.0,5.00,12,1080.0,0.061073,...,145.000001,401.347252,245.050003,0.386370,1420.710954,2401.001512,5.802420,3.550,0.0,0.0


# Zapisanie wyników do pliku

In [ ]:
df.to_excel('Machine_Learning_dataframe.xlsx')
print("Plik o nazwie: Machine_Learning_dataframe.xlsx został utworzony")